# Tiago's Coursera Capstone Notebook

#### 1. Hello, this notebook is part of my journey at the Applied Data Science Capstone Project at Coursera. Find more at: https://www.coursera.org/learn/applied-data-science-capstone/home/welcome

### PART1

In [47]:
### 2. Imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml
import requests
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium


In [2]:
#retrive the webpage as requested.
pagina = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(pagina)

In [3]:
# Expecting 200 OK
resp


<Response [200]>

In [4]:
# Parsing with BeatifulSoup
soup = BeautifulSoup(resp.text, 'lxml')
#print(soup.table.text)
#print(soup.table)

In [5]:
#soup.table

In [6]:
#print(soup.prettify())
df = pd.DataFrame
#df = soup.table.text
df = pd.read_html(str(soup.table))
df = df[0]

In [7]:
#removing "Not assigned Borough's"
df = df.loc[:][df['Borough']!='Not assigned'] 
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [8]:
df.loc[:][df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [9]:
# Setting Neighbourhood = Borough, if Neighbourhood = 'Not assigned'
df.loc[df['Neighbourhood'] == 'Not assigned',['Borough','Neighbourhood']]  = df['Borough']


In [10]:
# Expecting a null list
df.loc[:][df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


In [11]:
df.shape

(211, 3)

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda tags: ','.join(tags))

In [14]:
df = pd.DataFrame(df)

In [15]:
df.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [16]:
#Finally, the DataFrame expected
df.reset_index(inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 

In [17]:
df[df.loc[:]['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"


In [18]:
df.shape

(103, 3)

### END OF PART1 

### PART2

In [19]:
df['lat'] = df['long'] = np.nan

In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

for index, row in df.iterrows():
    location = str(df.loc[index]['Borough'] +',' + df.loc[index]['Postcode']+', Canada')
    result = geolocator.geocode(location)
    try:
        df['lat'].loc[index] = result.latitude
        df['long'].loc[index] = result.longitude
    except:
        pass

/home/tblacerda/aplicativos/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


GeocoderTimedOut: Service timed out

In [21]:
df.head()

,Postcode,Borough,Neighbourhood,lat,long
0,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.773077,-79.257774
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,43.762669,-79.230861
4,M1H,Scarborough,Cedarbrae,NaN,NaN


#### As seen above, geocoders could not convert all zip-codes + Boroughs to geographical coordinates. So, I had
#### to use the CSV file provided. 

In [22]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')

In [23]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
### Using the merge procedure to grab coordinates from postal codes.
df = pd.merge(left = df, right = df_coord, how='left', left_on='Postcode', right_on = 'Postal Code')

In [25]:
df.head()

,Postcode,Borough,Neighbourhood,lat,long,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774,M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.773077,-79.257774,M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",NaN,NaN,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.762669,-79.230861,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,NaN,NaN,M1H,43.773136,-79.239476


In [26]:
df.drop(['lat', 'long', 'Postal Code'], axis = 1, inplace=True)

### Final Result

In [27]:
### RESULT
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [72]:
df.shape 

(103, 5)

## Exploring the neighbourhood with Clusttering

In [73]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canada are 43.653963, -79.387207.


In [74]:
### 1 - Create a MAP.
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [82]:
### Foursquare Credentials

CLIENT_ID = '1GU25OCTP5HKPFYDDYM1W2T3Q4VK0ID0XB3EJYUQ0BDZPL1D' # your Foursquare ID
CLIENT_SECRET = 'blank' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1GU25OCTP5HKPFYDDYM1W2T3Q4VK0ID0XB3EJYUQ0BDZPL1D
CLIENT_SECRET:NSJW03UNRYXOHZYYS4HQSUWFOMLUW0D2F24MCXFAV1H0NBB2


In [83]:
### repeating for all points


In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [86]:
canada_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [87]:
print(canada_venues.shape)
canada_venues.head(10)

(2231, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [88]:
canada_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [90]:
print('There are {} uniques categories.'.format(len(canada_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [91]:
### Analysing each neighborhood
# one hot encoding
canada_onehot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_onehot['Neighborhood'] = canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]

canada_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [93]:
num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2    Skating Rink  0.25
3  Clothing Store  0.25
4  Medical Center  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                             Park   0.5
1                       Playground   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1  Fast Food Restaurant  0.11
2        Sandwich Place  0.11
3   Fried Chicken Joint  0.11
4            Beer Store  0.11


----Alderwood,Long Branch----
            venue  f

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Hotel,Asian Restaurant,Gym,American Restaurant,Breakfast Spot
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Pizza Place,Sandwich Place,Doner Restaurant,Diner
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Dance Studio,Pharmacy,Gym,Comic Shop,Electronics Store


In [96]:
# set number of clusters
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [97]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [98]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merge = canada_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merge = canada_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

canada_merge.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,3,Fast Food Restaurant,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hakka Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,0,Moving Target,Bar,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target,0,Moving Target,Bar,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar


In [102]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_merge['Neighborhood Latitude'], canada_merge['Neighborhood Longitude'], 
                                  canada_merge['Neighborhood'], canada_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [103]:
### Examining the clusters


In [104]:
## Cluster 1
canada_merge.loc[canada_merge['Cluster Labels'] == 0, canada_merge.columns[[1] + 
                                                                           list(range(5, canada_merge.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,43.784535,-79.163085,Bar,0,Moving Target,Bar,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,43.784535,-79.162977,Moving Target,0,Moving Target,Bar,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,43.763573,-79.189914,Pizza Place,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
4,43.763573,-79.191537,Electronics Store,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
5,43.763573,-79.190720,Mexican Restaurant,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
6,43.763573,-79.193406,Rental Car Location,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
7,43.763573,-79.192286,Medical Center,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
8,43.763573,-79.189490,Intersection,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
9,43.763573,-79.190466,Breakfast Spot,0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
10,43.770992,-79.221156,Coffee Shop,0,Coffee Shop,Korean Restaurant,Pharmacy,Women's Store,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [105]:
### Cluster 2
canada_merge.loc[canada_merge['Cluster Labels'] == 1, canada_merge.columns[[1] + 
                                                                    list(range(5, canada_merge.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,43.815252,-79.289773,Park,1,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
68,43.815252,-79.289867,Playground,1,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
201,43.753259,-79.332140,Park,1,Fast Food Restaurant,Food & Drink Shop,Park,Grocery Store,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
202,43.753259,-79.333021,Fast Food Restaurant,1,Fast Food Restaurant,Food & Drink Shop,Park,Grocery Store,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
203,43.753259,-79.333114,Food & Drink Shop,1,Fast Food Restaurant,Food & Drink Shop,Park,Grocery Store,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
255,43.737473,-79.470111,Airport,1,Park,Airport,Bus Stop,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
256,43.737473,-79.464221,Bus Stop,1,Park,Airport,Bus Stop,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
257,43.737473,-79.464777,Park,1,Park,Airport,Bus Stop,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
352,43.685347,-79.333846,Pizza Place,1,Park,Pizza Place,Coffee Shop,Convenience Store,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
353,43.685347,-79.341091,Park,1,Park,Pizza Place,Coffee Shop,Convenience Store,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop


In [106]:
### Cluster 3
canada_merge.loc[canada_merge['Cluster Labels'] == 2, canada_merge.columns[[1] + 
                                                                    list(range(5, canada_merge.shape[1]))]]


,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
156,43.75749,-79.370649,Cafeteria,2,Cafeteria,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Women's Store,College Stadium


In [107]:
### Cluster 4
canada_merge.loc[canada_merge['Cluster Labels'] == 3, canada_merge.columns[[1] + 
                                                                    list(range(5, canada_merge.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.806686,-79.199056,Fast Food Restaurant,3,Fast Food Restaurant,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hakka Restaurant


In [108]:
### Cluster 4
canada_merge.loc[canada_merge['Cluster Labels'] == 4, canada_merge.columns[[1] + 
                                                                    list(range(5, canada_merge.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1480,43.711695,-79.420702,Home Service,4,Home Service,Garden,Music Venue,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
1481,43.711695,-79.411978,Garden,4,Home Service,Garden,Music Venue,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
1482,43.711695,-79.411680,Music Venue,4,Home Service,Garden,Music Venue,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
2205,43.756303,-79.560256,Home Service,4,Home Service,Empanada Restaurant,Women's Store,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2206,43.756303,-79.570637,Empanada Restaurant,4,Home Service,Empanada Restaurant,Women's Store,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
